# SimCLR Notebook

## Torch Setup

In [ ]:
import torch
from torch import device, nn
from torch.utils.data import DataLoader
import torchvision
import torchvision.io as io

torch.manual_seed(42)

print(f'PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}')

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Using device: {device}')

## Data Setup

In [ ]:
from utils.clr_dataset import ContrastiveLearningDataset

out_dim = 128
temperature = 0.07
learning_rate = 0.0003
weight_decay = 1e-4
contrastive_views = 2
BATCH_SIZE = 256

dataset = ContrastiveLearningDataset('data/cifar-10/')
train_dataset = dataset.get_dataset('cifar10', contrastive_views)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

## Model Setup

In [ ]:
from model.chrome_CLR import ChromeCLR
from model.CLR_trainer import CLRTrainer

model = ChromeCLR(out_dim=out_dim)
#optimizer = torch.optim.Adam(model.parameters(), learning_rate, weight_decay)
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader), eta_min=0, last_epoch=-1)
criterion = torch.nn.CrossEntropyLoss().to(device)

## Training

In [ ]:
EPOCHS = 10

trainer = CLRTrainer(epochs=EPOCHS, model=model, optimizer=optimizer, scheduler=scheduler, criterion=criterion, batch_size=BATCH_SIZE, device=device, temp=temperature, con_views=contrastive_views)

trainer.train(train_loader=train_loader)